<a href="https://colab.research.google.com/github/cleomarbrdias/script/blob/main/baixa_x_pco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# 🟩 Validação: títulos baixados (numero) que NÃO aparecem no PCO com Saldo = EF
# Corrigido: comparação de dígitos com dígitos, sem mostrar títulos encontrados

!pip -q install pandas openpyxl xlsxwriter

import re
import pandas as pd
from google.colab import files

# ---------------------------
# 1) Upload
# ---------------------------
print("Envie primeiro: Planilha de Baixas (.xlsx)")
up1 = files.upload()
baixas_name = list(up1.keys())[0]

print("\nAgora envie: Planilha PCO (.xlsx)")
up2 = files.upload()
pco_name = list(up2.keys())[0]

# ---------------------------
# 2) Funções auxiliares
# ---------------------------
def pick_sheet(xls: pd.ExcelFile, preferred_keywords):
    for n in xls.sheet_names:
        low = n.lower()
        if any(k in low for k in preferred_keywords):
            return n
    return xls.sheet_names[0]

def s(x):
    if pd.isna(x): return ""
    return str(x).strip()

def only_digits(text: str) -> str:
    return re.sub(r"\D+", "", text or "")

def detect_col(df, candidates, required=True):
    m = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in m:
            return m[cand.lower()]
    if required:
        raise ValueError(f"Não encontrei {candidates}. Colunas: {list(df.columns)}")
    return None

# ---------------------------
# 3) Ler e escolher abas
# ---------------------------
xls_bx = pd.ExcelFile(baixas_name)
xls_pco = pd.ExcelFile(pco_name)

baixas_sheet = pick_sheet(xls_bx, ["baixa","baixas","setembro","baixar"])
pco_sheet    = pick_sheet(xls_pco, ["pco","ultima","listagem","browse","lanc","mov"])

print(f"\nAba (Baixas): {baixas_sheet}")
print(f"Aba (PCO)   : {pco_sheet}")

df_bx  = pd.read_excel(baixas_name, sheet_name=baixas_sheet, dtype=str).applymap(s)
df_pco = pd.read_excel(pco_name,    sheet_name=pco_sheet,    dtype=str).applymap(s)

# ---------------------------
# 4) Colunas
# ---------------------------
col_num_bx = detect_col(df_bx,  ["numero","número","titulo","título"])
col_hist   = detect_col(df_pco, ["Historico","Histórico","historico","histórico"])
col_saldo  = detect_col(df_pco, ["Saldo","saldo"])

# ---------------------------
# 5) Preparar
# ---------------------------
df_bx = df_bx.reset_index().rename(columns={"index":"_row0"})
df_bx["linha_excel_baixas"] = df_bx["_row0"] + 2
df_bx["_numero_digits"] = df_bx[col_num_bx].apply(only_digits)

df_pco = df_pco.reset_index().rename(columns={"index":"_row0"})
df_pco["linha_excel_pco"] = df_pco["_row0"] + 2
df_pco["_saldo_EF"] = df_pco[col_saldo].str.upper().str.strip().eq("EF")
df_pco["_historico_digits"] = df_pco[col_hist].apply(only_digits)

# ---------------------------
# 6) Avaliação: “não localizado com EF”
# ---------------------------
nao_localizados = []
historicos_dig = df_pco["_historico_digits"].fillna("")

for _, r in df_bx.iterrows():
    raw_num   = r[col_num_bx]
    num_dig   = r["_numero_digits"]

    if not num_dig:
        nao_localizados.append({
            "linha_excel_baixas": r["linha_excel_baixas"],
            "numero": raw_num,
            "motivo": "numero sem dígitos"
        })
        continue

    # linhas do PCO onde historico_digits contém numero_digits
    mask_any = historicos_dig.str.contains(num_dig, na=False)

    if not mask_any.any():
        nao_localizados.append({
            "linha_excel_baixas": r["linha_excel_baixas"],
            "numero": raw_num,
            "motivo": "não encontrado no Historico do PCO"
        })
        continue

    # requer ao menos 1 ocorrência EF
    mask_ef = mask_any & df_pco["_saldo_EF"]
    if not mask_ef.any():
        nao_localizados.append({
            "linha_excel_baixas": r["linha_excel_baixas"],
            "numero": raw_num,
            "motivo": "encontrado no PCO, porém sem Saldo = EF"
        })

# ---------------------------
# 7) Exportar resultado
# ---------------------------
df_nao_loc = (
    pd.DataFrame(nao_localizados)
      .sort_values(["numero","linha_excel_baixas"])
      .drop_duplicates(subset=["numero"], keep="first")
)

print("\n=== RESUMO ===")
print(f"Total de títulos (baixas): {len(df_bx)}")
print(f"Títulos NÃO localizados no PCO com EF: {len(df_nao_loc)}")

display(df_nao_loc)

out_name = "titulos_nao_localizados_no_PCO_pelo_SISTEMA.xlsx"
with pd.ExcelWriter(out_name, engine="xlsxwriter") as w:
    df_nao_loc.to_excel(w, index=False, sheet_name="Nao_localizados_pelo_SISTEMA")

print(f"\nArquivo gerado: {out_name}")
files.download(out_name)


Envie primeiro: Planilha de Baixas (.xlsx)


Saving Planilha de Baixas - Setembro 2025.xlsx to Planilha de Baixas - Setembro 2025 (8).xlsx

Agora envie: Planilha PCO (.xlsx)


Saving pcoa050-01-10.xlsx to pcoa050-01-10 (3).xlsx

Aba (Baixas): baixas
Aba (PCO)   : 01-01 - Listagem do Browse


/tmp/ipython-input-1661538669.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_bx  = pd.read_excel(baixas_name, sheet_name=baixas_sheet, dtype=str).applymap(s)
/tmp/ipython-input-1661538669.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pco = pd.read_excel(pco_name,    sheet_name=pco_sheet,    dtype=str).applymap(s)



=== RESUMO ===
Total de títulos (baixas): 218
Títulos NÃO localizados no PCO com EF: 26


,linha_excel_baixas,numero,motivo
28,161,1178,"encontrado no PCO, porém sem Saldo = EF"
21,107,16424,"encontrado no PCO, porém sem Saldo = EF"
5,15,183812,"encontrado no PCO, porém sem Saldo = EF"
10,53,183996,"encontrado no PCO, porém sem Saldo = EF"
11,65,20104564,não encontrado no Historico do PCO
12,66,20104565,não encontrado no Historico do PCO
19,101,20104566,não encontrado no Historico do PCO
17,72,22254,"encontrado no PCO, porém sem Saldo = EF"
2,4,39668,"encontrado no PCO, porém sem Saldo = EF"
3,5,39676,"encontrado no PCO, porém sem Saldo = EF"



Arquivo gerado: titulos_nao_localizados_no_PCO_pelo_SISTEMA.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>